In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import os
from sklearn.preprocessing import LabelEncoder 
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [2]:
le = LabelEncoder()

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

## Read in Data and preform basic cleaning 


In [4]:
file_path = Path("Resources/world_2019_data.csv")

In [5]:
df = pd.read_csv(file_path)
df

,country_name,country_code,h_2019,edu_2019,e_2019,con_2019,inf_2019,une_2019,p_2019,g_2019,income_group
0,Angola,AGO,2.53,2.923862e+09,2.827171e+10,4.057062e+10,17.08,7.42,31825299,-0.70,Low income
1,Armenia,ARM,11.34,3.161166e+08,5.631686e+09,1.307507e+10,1.44,18.30,2957728,7.60,High income
2,Australia,AUS,9.91,6.777152e+10,3.364310e+11,1.033080e+12,1.61,5.16,25365745,2.11,High income
3,Austria,AUT,10.43,2.241872e+10,2.467240e+11,3.160610e+11,1.53,4.49,8879920,1.49,High income
4,Azerbaijan,AZE,4.04,1.178561e+09,2.362976e+10,3.306206e+10,2.61,4.85,10024283,2.50,High income
...,...,...,...,...,...,...,...,...,...,...,...
123,Ukraine,UKR,7.10,7.662706e+09,6.344856e+10,1.433540e+11,7.89,8.19,44386203,3.20,Low income
124,Uruguay,URY,9.35,2.774959e+09,1.699176e+10,4.865871e+10,7.88,8.88,3461731,0.35,High income
125,United States,USA,16.77,9.540000e+11,2.519730e+12,1.740260e+13,1.81,3.67,328329953,2.29,High income
126,Vanuatu,VUT,3.36,4.506711e+07,4.714967e+08,7.126577e+08,2.76,1.80,299882,3.24,Low income


In [6]:
# #drop string columns
# df = df.drop(columns=["country_name", "country_code"])
# df

In [7]:
#check data types
df.dtypes

country_name     object
country_code     object
h_2019          float64
edu_2019        float64
e_2019          float64
con_2019        float64
inf_2019        float64
une_2019        float64
p_2019            int64
g_2019          float64
income_group     object
dtype: object

In [8]:
#encode y column
df['income_group'] = le.fit_transform(df['income_group'])
df = df.drop(columns=["country_name", "country_code"])

In [9]:
#define Varibles
X = df.drop(columns= "income_group")
X


,h_2019,edu_2019,e_2019,con_2019,inf_2019,une_2019,p_2019,g_2019
0,2.53,2.923862e+09,2.827171e+10,4.057062e+10,17.08,7.42,31825299,-0.70
1,11.34,3.161166e+08,5.631686e+09,1.307507e+10,1.44,18.30,2957728,7.60
2,9.91,6.777152e+10,3.364310e+11,1.033080e+12,1.61,5.16,25365745,2.11
3,10.43,2.241872e+10,2.467240e+11,3.160610e+11,1.53,4.49,8879920,1.49
4,4.04,1.178561e+09,2.362976e+10,3.306206e+10,2.61,4.85,10024283,2.50
...,...,...,...,...,...,...,...,...
123,7.10,7.662706e+09,6.344856e+10,1.433540e+11,7.89,8.19,44386203,3.20
124,9.35,2.774959e+09,1.699176e+10,4.865871e+10,7.88,8.88,3461731,0.35
125,16.77,9.540000e+11,2.519730e+12,1.740260e+13,1.81,3.67,328329953,2.29
126,3.36,4.506711e+07,4.714967e+08,7.126577e+08,2.76,1.80,299882,3.24


In [10]:
y = df["income_group"]
y

0      1
1      0
2      0
3      0
4      0
      ..
123    1
124    0
125    0
126    1
127    0
Name: income_group, Length: 128, dtype: int32

In [11]:
# Check the balance of our target values
y.value_counts()

0    79
1    49
Name: income_group, dtype: int64

In [12]:
y_percent = (49/(79+49))*100
print(f"y = {y_percent}% of training group")

y = 38.28125% of training group


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
#check the size of the training and testing sets
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(96, 8)
(32, 8)
(96,)
(32,)


In [15]:
# # Standardize the data with StandardScaler().
# income_scaled = StandardScaler().fit_transform(df)
# print(income_scaled)



# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Random Forest Classifier


In [16]:
# Resample the training data with the BalancedRandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=1)


In [17]:
#fit the model
random_forest = random_forest.fit(X_train, y_train)

In [18]:
#calculate balanced accuracy score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.825

In [19]:
#create the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[18,  2],
       [ 3,  9]], dtype=int64)

In [20]:
#display the confusion matrix
cm_df = pd.DataFrame(cm)
cm_df = cm_df.set_axis(["High Income", "Low Income"], axis = 1)
cm_df = cm_df.set_axis(["High Income", "Low Income"], axis = 0)
cm_df

,High Income,Low Income
High Income,18,2
Low Income,3,9


In [21]:
#Print imbalanced classification report
rf_imbalanced_classification_report = classification_report(y_test, y_pred)
print(rf_imbalanced_classification_report)

              precision    recall  f1-score   support

           0       0.86      0.90      0.88        20
           1       0.82      0.75      0.78        12

    accuracy                           0.84        32
   macro avg       0.84      0.82      0.83        32
weighted avg       0.84      0.84      0.84        32



In [22]:
# Calculate feature importance in the Random Forest model.
# importances = random_forest.feature_importances_
# importances

# sort by importance
rf_importances = sorted(zip(random_forest.feature_importances_, X.columns), reverse=True)
rf_importances

[(0.20263581061361444, 'e_2019'),
 (0.1508718466995432, 'h_2019'),
 (0.14922838270663585, 'inf_2019'),
 (0.13270022316218957, 'p_2019'),
 (0.12702210445601658, 'edu_2019'),
 (0.09074350099131952, 'g_2019'),
 (0.0891304630718977, 'con_2019'),
 (0.05766766829878314, 'une_2019')]

In [23]:
#Summarize importances using a df
#a. read into df
rf_features_importances_df = pd.DataFrame(rf_importances)

# b. rename columns
rf_features_importances_df = rf_features_importances_df.set_axis(["Importance", "Indicator"], axis=1)
# print(rf_features_importances_df)

#c.replace abbreviations with category string
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('e_2019',"Exports", case=False)
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('h_2019',"Healthcare Expenditures", case=False)
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('inf_2019',"Inflation Rate", case=False)
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('p_2019',"Population", case=False)
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('edu_2019',"Education Expenditures", case=False)
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('g_2019',"GDP Growth", case=False)
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('con_2019',"Consumption", case=False)
rf_features_importances_df["Indicator"] = rf_features_importances_df["Indicator"].str.replace('une_2019',"Unemployment", case=False)
rf_features_importances_df

,Importance,Indicator
0,0.202636,Exports
1,0.150872,Healthcare Expenditures
2,0.149228,Inflation Rate
3,0.132700,Population
4,0.127022,Education Expenditures
5,0.090744,GDP Growth
6,0.089130,Consumption
7,0.057668,unExports


In [24]:
#summary df rounded
rounded_rf_features_importances_df = rf_features_importances_df.round(2)
rounded_rf_features_importances_df

,Importance,Indicator
0,0.20,Exports
1,0.15,Healthcare Expenditures
2,0.15,Inflation Rate
3,0.13,Population
4,0.13,Education Expenditures
5,0.09,GDP Growth
6,0.09,Consumption
7,0.06,unExports


## Balanced Random Forest

In [25]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
balanced_forest = BalancedRandomForestClassifier(n_estimators=100, random_state=1)


In [26]:
#fit the model
balanced_forest = balanced_forest.fit(X_train, y_train)

In [27]:
#calculate balanced accuracy score
y_pred = balanced_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.8

In [28]:
#display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[17,  3],
       [ 3,  9]], dtype=int64)

In [29]:
cm_df = pd.DataFrame(cm)
cm_df = cm_df.set_axis(["High Income", "Low Income"], axis = 1)
cm_df = cm_df.set_axis(["High Income", "Low Income"], axis = 0)
cm_df

,High Income,Low Income
High Income,17,3
Low Income,3,9


In [30]:
#Print imbalanced classification report
brf_imbalanced_classification_report = classification_report(y_test, y_pred)
print(brf_imbalanced_classification_report)

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        20
           1       0.75      0.75      0.75        12

    accuracy                           0.81        32
   macro avg       0.80      0.80      0.80        32
weighted avg       0.81      0.81      0.81        32



In [31]:
# Calculate feature importance in the Random Forest model.
# importances = random_forest.feature_importances_
# importances

# sort by importance
brf_importances = sorted(zip(balanced_forest.feature_importances_, X.columns), reverse=True)
brf_importances

[(0.2139353244023755, 'e_2019'),
 (0.15567810035383087, 'h_2019'),
 (0.13979544771317348, 'inf_2019'),
 (0.12907788786821772, 'p_2019'),
 (0.1268409930368637, 'edu_2019'),
 (0.09429433258179466, 'con_2019'),
 (0.08265137475137282, 'g_2019'),
 (0.05772653929237132, 'une_2019')]

In [32]:
#Summarize importances using a df
#a. read into df
brf_features_importances_df = pd.DataFrame(brf_importances)
brf_features_importances_df

# b. rename columns
brf_features_importances_df = brf_features_importances_df.set_axis(["Importance", "Indicator"], axis=1)
# print(brf_features_importances_df)

#c.replace abbreviations with category string
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('e_2019',"Exports", case=False)
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('h_2019',"Healthcare Expenditures", case=False)
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('inf_2019',"Inflation Rate", case=False)
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('p_2019',"Population", case=False)
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('edu_2019',"Education Expenditures", case=False)
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('g_2019',"GDP Growth", case=False)
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('con_2019',"Consumption", case=False)
brf_features_importances_df["Indicator"] = brf_features_importances_df["Indicator"].str.replace('une_2019',"Unemployment", case=False)



## Summary

In [33]:
#1. Random Forest
print("Imbalanced Classification Report")
print(rf_imbalanced_classification_report)
rf_features_importances_df



Imbalanced Classification Report
              precision    recall  f1-score   support

           0       0.86      0.90      0.88        20
           1       0.82      0.75      0.78        12

    accuracy                           0.84        32
   macro avg       0.84      0.82      0.83        32
weighted avg       0.84      0.84      0.84        32



,Importance,Indicator
0,0.202636,Exports
1,0.150872,Healthcare Expenditures
2,0.149228,Inflation Rate
3,0.132700,Population
4,0.127022,Education Expenditures
5,0.090744,GDP Growth
6,0.089130,Consumption
7,0.057668,unExports


In [34]:
#2. Balanced Random Forest
print("Imbalanced Classification Report")
print(brf_imbalanced_classification_report)
brf_features_importances_df


Imbalanced Classification Report
              precision    recall  f1-score   support

           0       0.85      0.85      0.85        20
           1       0.75      0.75      0.75        12

    accuracy                           0.81        32
   macro avg       0.80      0.80      0.80        32
weighted avg       0.81      0.81      0.81        32



,Importance,Indicator
0,0.213935,Exports
1,0.155678,Healthcare Expenditures
2,0.139795,Inflation Rate
3,0.129078,Population
4,0.126841,Education Expenditures
5,0.094294,Consumption
6,0.082651,GDP Growth
7,0.057727,unExports
